# Criando Cópias de segurança e Restaurando Bancos de dados

### Criando banco de dados

In [1]:
import psycopg2
conn = psycopg2.connect(host="localhost",database="postgres", user="postgres", password="123456")
cur = conn.cursor()
cur.execute('commit')
cur.execute("drop database if exists bk_estudo")
cur.execute("""
    CREATE DATABASE bk_estudo
    WITH 
    OWNER = postgres
    ENCODING = 'UTF8'
    CONNECTION LIMIT = -1;
    """)
conn.commit()
conn.close()

### Criando tabela e registros de exemplo

In [2]:
import psycopg2
conn = psycopg2.connect(host="localhost",database="bk_estudo", user="postgres", password="123456")
cur = conn.cursor()
cur.execute("start transaction")
cur.execute("""
            create table table_bk_estudo 
            (cod_bk integer primary key,
            nome_bk varchar(80)); """)
cur.execute(""" insert into table_bk_estudo 
            values  (1,'nome01_bk'),
                    (2,'nome02_bk'),
                    (3,'nome03_bk');""")
cur.execute("commit")
cur.close()
conn.close()

# Criando aquivo de backup no formato plain text

In [3]:
!PGPASSWORD=123456 pg_dump --host=localhost --user=postgres --dbname=bk_estudo> bk_estudo/bk_estudo_plain.sql

In [4]:
!ls -l bk_estudo/

total 4
-rw-r--r-- 1 dbaprof dbaprof 1516 set  8 20:19 bk_estudo_plain.sql


#### Outro exemplo utilizando variaveis de ambiente do postgres

In [5]:
!PGHOST=localhost PGPORT=5432  PGDATABASE=bk_estudo  PGUSER=postgres PGPASSWORD=123456 \
pg_dump > bk_estudo/bk_estudo_plain.sql

# Verificando arquivo criado e conteúdo SQL

In [6]:
!ls  bk_estudo/

bk_estudo_plain.sql


In [7]:
!cat bk_estudo/bk_estudo_plain.sql

--
-- PostgreSQL database dump
--

-- Dumped from database version 10.12 (Ubuntu 10.12-0ubuntu0.18.04.1)
-- Dumped by pg_dump version 10.12 (Ubuntu 10.12-0ubuntu0.18.04.1)

SET statement_timeout = 0;
SET lock_timeout = 0;
SET idle_in_transaction_session_timeout = 0;
SET client_encoding = 'UTF8';
SET standard_conforming_strings = on;
SELECT pg_catalog.set_config('search_path', '', false);
SET check_function_bodies = false;
SET xmloption = content;
SET client_min_messages = warning;
SET row_security = off;

--
-- Name: plpgsql; Type: EXTENSION; Schema: -; Owner: 
--

CREATE EXTENSION IF NOT EXISTS plpgsql WITH SCHEMA pg_catalog;


--
-- Name: EXTENSION plpgsql; Type: COMMENT; Schema: -; Owner: 
--

COMMENT ON EXTENSION plpgsql IS 'PL/pgSQL procedural language';


SET default_tablespace = '';

SET default_with_oids = false;

--
-- Name: table_bk_estudo; Type: TABLE; Schema: public; Owner: postgres
--

CREATE TABLE public.table_bk_estudo (
    cod_bk integer NOT NULL,
    nome_bk character

### O parâmetro "--data-only" pode ser utilizado para obter somente o conjunto de dados e o parametro --schema-only pode ser utilizado para obtenção da estrutura do database.

In [8]:
!PGHOST=localhost PGPORT=5432  PGDATABASE=bk_estudo  PGUSER=postgres PGPASSWORD=123456 \
pg_dump --data-only --file='bk_estudo/bk_estudo_data_only.sql'

In [9]:
!PGHOST=localhost PGPORT=5432  PGDATABASE=bk_estudo  PGUSER=postgres PGPASSWORD=123456 \
pg_dump --schema-only --file='bk_estudo/bk_estudo_schema_only.sql'

In [10]:
!ls -l --sort=time bk_estudo/

total 12
-rw-r--r-- 1 dbaprof dbaprof 1325 set  8 20:21 bk_estudo_schema_only.sql
-rw-r--r-- 1 dbaprof dbaprof  746 set  8 20:21 bk_estudo_data_only.sql
-rw-r--r-- 1 dbaprof dbaprof 1516 set  8 20:20 bk_estudo_plain.sql


# Vamos comparar as diferenças entre os aquivos obtidos

In [11]:
!diff -y bk_estudo/bk_estudo_data_only.sql bk_estudo/bk_estudo_schema_only.sql

--								--
-- PostgreSQL database dump					-- PostgreSQL database dump
--								--

-- Dumped from database version 10.12 (Ubuntu 10.12-0ubuntu0.	-- Dumped from database version 10.12 (Ubuntu 10.12-0ubuntu0.
-- Dumped by pg_dump version 10.12 (Ubuntu 10.12-0ubuntu0.18.	-- Dumped by pg_dump version 10.12 (Ubuntu 10.12-0ubuntu0.18.

SET statement_timeout = 0;					SET statement_timeout = 0;
SET lock_timeout = 0;						SET lock_timeout = 0;
SET idle_in_transaction_session_timeout = 0;			SET idle_in_transaction_session_timeout = 0;
SET client_encoding = 'UTF8';					SET client_encoding = 'UTF8';
SET standard_conforming_strings = on;				SET standard_conforming_strings = on;
SELECT pg_catalog.set_config('search_path', '', false);		SELECT pg_catalog.set_config('search_path', '', false);
SET check_function_bodies = false;				SET check_function_bodies = false;
SET xmloption = content;					SET xmloption = content;
SET client_min_messages = warning;				SET client_min_messages = warning;
SET row_s

# Utilizando a opção verbose 

# Criando aquivo de backup no formato custom e aplicando opção clean (realiza drop antes de reconstruir a base) 

In [12]:
!PGHOST=localhost PGPORT=5432  PGDATABASE=bk_estudo  PGUSER=postgres PGPASSWORD=123456 \
pg_dump --clean --verbose --format=custom --file='bk_estudo/bk_estudo_clean_dump.custom'

pg_dump: last built-in OID is 16383
pg_dump: reading extensions
pg_dump: identifying extension members
pg_dump: reading schemas
pg_dump: reading user-defined tables
pg_dump: reading user-defined functions
pg_dump: reading user-defined types
pg_dump: reading procedural languages
pg_dump: reading user-defined aggregate functions
pg_dump: reading user-defined operators
pg_dump: reading user-defined access methods
pg_dump: reading user-defined operator classes
pg_dump: reading user-defined operator families
pg_dump: reading user-defined text search parsers
pg_dump: reading user-defined text search templates
pg_dump: reading user-defined text search dictionaries
pg_dump: reading user-defined text search configurations
pg_dump: reading user-defined foreign-data wrappers
pg_dump: reading user-defined foreign servers
pg_dump: reading default privileges
pg_dump: reading user-defined collations
pg_dump: reading user-defined conversions
pg_dump: reading type casts
pg_dump: reading transforms
pg_d

# Analisando NJobs e tempo de processamento (utilizando uma base com mais registros e tabelas)

In [13]:
from datetime import datetime 
start_time = datetime.now() 
#-----------------------------------------------------------------------------------------
!PGHOST=localhost PGPORT=5432  PGDATABASE=dw_estudo2  PGUSER=postgres PGPASSWORD=123456 \
pg_dump --format=custom --file='bk_estudo/dw_estudo2_file.custom'
#-----------------------------------------------------------------------------------------
end_time = datetime.now() 
print('Duration: {}'.format(end_time - start_time))


Duration: 0:00:02.705846


In [14]:
from datetime import datetime 
start_time = datetime.now() 
#-----------------------------------------------------------------------------------------
!PGHOST=localhost PGPORT=5432  PGDATABASE=dw_estudo2  PGUSER=postgres PGPASSWORD=123456 \
pg_dump --format=directory --jobs=8 --file='bk_estudo/dw_estudo2_directory_jobs'
#-----------------------------------------------------------------------------------------
end_time = datetime.now() 
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:02.569895


In [15]:
!ls -l --sort=time --block-size=k 'bk_estudo/dw_estudo2_directory_jobs'

total 5200K
-rw-r--r-- 1 dbaprof dbaprof 5100K set  8 20:24 2945.dat.gz
-rw-r--r-- 1 dbaprof dbaprof   80K set  8 20:24 2949.dat.gz
-rw-r--r-- 1 dbaprof dbaprof    1K set  8 20:24 2947.dat.gz
-rw-r--r-- 1 dbaprof dbaprof    4K set  8 20:24 2951.dat.gz
-rw-r--r-- 1 dbaprof dbaprof   12K set  8 20:24 toc.dat


# Acessando opções disponíveis com o --help

In [16]:
!pg_dump --help

pg_dump dumps a database as a text file or to other formats.

Usage:
  pg_dump [OPTION]... [DBNAME]

General options:
  -f, --file=FILENAME          output file or directory name
  -F, --format=c|d|t|p         output file format (custom, directory, tar,
                               plain text (default))
  -j, --jobs=NUM               use this many parallel jobs to dump
  -v, --verbose                verbose mode
  -V, --version                output version information, then exit
  -Z, --compress=0-9           compression level for compressed formats
  --lock-wait-timeout=TIMEOUT  fail after waiting TIMEOUT for a table lock
  --no-sync                    do not wait for changes to be written safely to disk
  -?, --help                   show this help, then exit

Options controlling the output content:
  -a, --data-only              dump only the data, not the schema
  -b, --blobs                  include large objects in dump
  -B, --no-blobs               exclude large objects in d

# Restaurando uma base de dados com PG_RESTORE

### Conectando no database e criando o banco de dados onde será feita restauração

In [17]:
import psycopg2
conn = psycopg2.connect(host="localhost",database="postgres", user="postgres", password="123456")
cur = conn.cursor()
cur.execute('commit')
cur.execute("drop database if exists bk_estudo_restore")
cur.execute("""
    CREATE DATABASE bk_estudo_restore
    WITH 
    OWNER = postgres
    ENCODING = 'UTF8'
    CONNECTION LIMIT = -1;
    """)
conn.close()

### Verificando o arquivos criados anteriormente

In [18]:
!ls -l bk_estudo/

total 5224
-rw-r--r-- 1 dbaprof dbaprof    2638 set  8 20:23 bk_estudo_clean_dump.custom
-rw-r--r-- 1 dbaprof dbaprof     746 set  8 20:21 bk_estudo_data_only.sql
-rw-r--r-- 1 dbaprof dbaprof    1516 set  8 20:20 bk_estudo_plain.sql
-rw-r--r-- 1 dbaprof dbaprof    1325 set  8 20:21 bk_estudo_schema_only.sql
drwx------ 2 dbaprof dbaprof    4096 set  8 20:24 dw_estudo2_directory_jobs
-rw-r--r-- 1 dbaprof dbaprof 5326319 set  8 20:24 dw_estudo2_file.custom


### Escolheremos o arquivo 'bk_estudo_clean_dump.custom' e também vamos aplicar a opção verbose para acompanhar-mos o resultado

In [19]:
!PGHOST=localhost PGPORT=5432  PGDATABASE=bk_estudo_restore  PGUSER=postgres PGPASSWORD=123456 \
pg_restore --dbname 'bk_estudo_restore' --verbose bk_estudo/bk_estudo_clean_dump.custom

pg_restore: connecting to database for restore
pg_restore: creating SCHEMA "public"
pg_restore: creating COMMENT "SCHEMA public"
pg_restore: creating EXTENSION "plpgsql"
pg_restore: creating COMMENT "EXTENSION plpgsql"
pg_restore: creating TABLE "public.table_bk_estudo"
pg_restore: processing data for table "public.table_bk_estudo"
pg_restore: creating CONSTRAINT "public.table_bk_estudo table_bk_estudo_pkey"
pg_restore: creating ACL "SCHEMA public"


### Conectando a base de dados onde ocorreu restauração e aplicando select para verificar os dados

In [20]:
import psycopg2
conn = psycopg2.connect(host="localhost",database="bk_estudo_restore", user="postgres", password="123456")
cur = conn.cursor()
cur.execute("select * from table_bk_estudo")
for i in cur:
    print(i)
conn.close()

(1, 'nome01_bk')
(2, 'nome02_bk')
(3, 'nome03_bk')
